## superset4sparql

https://github.com/fbelleau/superset4sparql/

In [6]:
import sys

In [27]:
!{sys.executable} -m pip install sqlparse sqlglot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.1/274.1 KB 315.7 kB/s eta 0:00:00 kB/s eta 0:00:01:01


## Parse SQL

In [63]:
sql_query = '''
select t1.a, t1.b 
from foo t1 where t1.x>10
'''

### SQLparse

- https://sqlparse.readthedocs.io/en/latest/

In [8]:
import sqlparse

In [64]:
parsed = sqlparse.parse(sql_query)

In [65]:
sql = parsed[0]

In [66]:
[(t.ttype, t.value.upper()) for t in sql.tokens]

[(Token.Text.Whitespace.Newline, '\n'),
 (Token.Keyword.DML, 'SELECT'),
 (Token.Text.Whitespace, ' '),
 (None, 'T1.A, T1.B'),
 (Token.Text.Whitespace, ' '),
 (Token.Text.Whitespace.Newline, '\n'),
 (Token.Keyword, 'FROM'),
 (Token.Text.Whitespace, ' '),
 (None, 'FOO T1'),
 (Token.Text.Whitespace, ' '),
 (None, 'WHERE T1.X>10\n')]

In [17]:
dir(sql.tokens[0])

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_get_repr_name',
 '_get_repr_value',
 'flatten',
 'has_ancestor',
 'is_child_of',
 'is_group',
 'is_keyword',
 'is_whitespace',
 'match',
 'normalized',
 'parent',
 'ttype',
 'value',
 'within']

### SQLglot

- https://github.com/tobymao/sqlglot

In [32]:
from sqlglot import exp, parse_one

In [67]:
expression_tree = parse_one(sql_query)

In [68]:
expression_tree

(SELECT expressions: 
  (COLUMN this: 
    (IDENTIFIER this: a, quoted: False), table: 
    (IDENTIFIER this: t1, quoted: False)), 
  (COLUMN this: 
    (IDENTIFIER this: b, quoted: False), table: 
    (IDENTIFIER this: t1, quoted: False)), from: 
  (FROM this: 
    (TABLE this: 
      (IDENTIFIER this: foo, quoted: False), alias: 
      (TABLEALIAS this: 
        (IDENTIFIER this: t1, quoted: False)))), where: 
  (WHERE this: 
    (GT this: 
      (COLUMN this: 
        (IDENTIFIER this: x, quoted: False), table: 
        (IDENTIFIER this: t1, quoted: False)), expression: 
      (LITERAL this: 10, is_string: False))))

In [69]:
for column in expression_tree.find_all(exp.Column):
    print(column.table, column.alias_or_name)

t1 a
t1 b
t1 x


In [70]:
column.table

't1'

In [71]:
tables = [t for t in expression_tree.find_all(exp.Table)]

In [72]:
tables[0].name

'foo'

In [73]:
tables[0].alias_or_name

't1'

In [57]:
dir(tables[0])

['__class__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_hash',
 '_meta',
 '_set_parent',
 '_to_s',
 '_type',
 'add_comments',
 'alias',
 'alias_or_name',
 'append',
 'arg_key',
 'arg_types',
 'args',
 'assert_is',
 'bfs',
 'catalog',
 'comments',
 'copy',
 'db',
 'depth',
 'dfs',
 'dump',
 'error_messages',
 'expression',
 'expressions',
 'find',
 'find_all',
 'find_ancestor',
 'flatten',
 'hashable_args',
 'is_int',
 'is_number',
 'is_star',
 'is_string',
 'iter_expressions',
 'key',
 'load',
 'meta',
 'name',
 'output_name',
 'parent',
 'parent_select',
 'parts',
 'pop',
 'replace',
 'root',
 'same_parent',
 'set',
 'sql',
 'text',
 'this',
 

In [1]:
sql_query = '''
SELECT enzyme.enzyme_name, COUNT(DISTINCT reaction_protein.protein) AS count, publication.publication_title
FROM reaction_enzyme
JOIN enzyme ON reaction_enzyme.enzyme = enzyme.enzyme_uri
JOIN reaction_protein ON reaction_enzyme.reaction = reaction_protein.reaction
JOIN publication ON publication.publication_uri IN (
  SELECT DISTINCT publication.publication_uri
  FROM enzyme
  JOIN publication ON publication.publication_uri IN (
    SELECT DISTINCT mentions.publication_uri
    FROM mentions
    WHERE mentions.enzyme_uri = enzyme.enzyme_uri
  )
)
GROUP BY enzyme.enzyme_name, publication.publication_title
ORDER BY count DESC
'''

In [2]:
def sql2sparql(sql_query):

    query = sql_query
    query = query.replace('SELECT t AS t ','SELECT ?t ')
    query = query.replace('SELECT p AS p ','SELECT ?p ')
    query = query.replace(' ASC ',' ')
    query = query.replace(' DESC ',' ')
    
    #query = query.replace('SELECT c AS c,        p AS p,        t AS t ','SELECT ?c ?p ?t ')
    query = query.replace('FROM   (select','WHERE {select').replace(') AS virtual_table','}')
     
    query = query.replace(' c AS c',' c').replace(' p AS p',' p').replace(' t AS t',' t')
    query = query.replace(' c, ',' ?c ').replace(' p, ',' ?p ').replace(' t, ',' ?t ')
    query = query.replace(' c ',' ?c ').replace(' p ',' ?p ').replace(' t ',' ?t ')

    query = query.replace('sum(c) AS "SUM(c)"',' (sum(?c) AS ?zSUM_c) ')
    query = query.replace('"SUM(c)"','?zSUM_c')
    
    
    
    if query.find('IN (') != -1:
        query = query.replace("WHERE ?p IN ('http://xmlns.com/foaf/0.1/name')",'FILTER(?p  = <http://xmlns.com/foaf/0.1/name>)}')
        query = query.replace("WHERE ?t IN ('http://bio2rdf.org/bibtex#Article')",'FILTER(?t  = <http://bio2rdf.org/bibtex#Article>)}')
        query = query.replace("WHERE {select",'WHERE {{select')

    return query   

In [6]:
sparql_query = sql2sparql(sql_query)

In [3]:
sparql_query = '''
PREFIX bio2rdf: <http://bio2rdf.org/>

SELECT ?enzyme (COUNT(DISTINCT ?protein) AS ?count) ?title
WHERE {
  ?reaction a bio2rdf:Reaction .
  ?reaction bio2rdf:reaction_enzyme ?enzyme .
  ?reaction bio2rdf:reaction_protein ?protein .
  
  {
    SELECT ?enzyme ?title
    WHERE {
      ?enzyme a bio2rdf:Enzyme .
      ?enzyme bio2rdf:enzyme_name ?name .
      ?publication a bio2rdf:Publication .
      ?publication bio2rdf:mentions ?enzyme .
      ?publication bio2rdf:publication_title ?title .
    }
  }
}
GROUP BY ?enzyme ?title
ORDER BY DESC(?count)
'''

In [4]:
from SPARQLWrapper import SPARQLWrapper

In [9]:
print(sparql_query)

PREFIX bio2rdf: <http://bio2rdf.org/>

SELECT ?enzyme (COUNT(DISTINCT ?protein) AS ?count) ?title
WHERE {
  ?reaction a bio2rdf:Reaction .
  ?reaction bio2rdf:reaction_enzyme ?enzyme .
  ?reaction bio2rdf:reaction_protein ?protein .
  
  {
    SELECT ?enzyme ?title
    WHERE {
      ?enzyme a bio2rdf:Enzyme .
      ?enzyme bio2rdf:enzyme_name ?name .
      ?publication a bio2rdf:Publication .
      ?publication bio2rdf:mentions ?enzyme .
      ?publication bio2rdf:publication_title ?title .
    }
  }
}
GROUP BY ?enzyme ?title
ORDER BY DESC(?count)


In [5]:
sparql = SPARQLWrapper('http://sparql.uniprot.org/sparql')

https://sparql.uniprot.org/.well-known/sparql-examples/